### calculate reaction fingerprints

In [1]:
from rdkit import Chem
import pandas as pd

from tqdm.notebook import tqdm
from tqdm.contrib.concurrent import process_map
from rdkit.Chem import AllChem

import numpy as np
from rdkit import DataStructs
import umap

RDKit WARNING: [18:59:30] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
data = pd.read_csv("../data_files/smiles_min_dist_natoms.csv")

In [4]:
data.head()

,Unnamed: 0,smiles,min_dist_all,natoms
0,0,COON,6.0,4
1,1,CONO,6.0,4
2,2,CON=O,7.0,4
3,3,NOCO,4.0,4
4,4,C1ONO1,5.0,4


In [5]:
def calc_morgan(smi):
    ## using radius = 2 to match ECFP4
    return AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smi),2, nBits=1024)

fps = process_map(calc_morgan, 
                  data.smiles, 
                  chunksize = 1000, 
                  max_workers = 4)

  0%|          | 0/222739 [00:00<?, ?it/s]

RDKit WARNING: [19:00:46] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.


In [6]:
am33 = calc_morgan("CCC(O)=O.CCN")

am33 = np.array([i for i in am33])

In [7]:
fps_expanded = []
for fp in tqdm(fps):
    fps_expanded.append([i for i in fp])

  0%|          | 0/222739 [00:00<?, ?it/s]

In [8]:
fps_expanded = np.array(fps_expanded)
rxn_fps = (fps_expanded-am33).astype("int8")

np.save("./acam33_rxn_fps_1024.npy",rxn_fps)